In [2]:
!pip install deep_translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 754.5 kB/s eta 0:00:00


In [3]:
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
from io import BytesIO
from urllib.parse import urljoin, urlparse
from concurrent.futures import ThreadPoolExecutor, as_completed
from PIL import Image
import os
import random
import re
import pandas as pd
import hashlib
from deep_translator import GoogleTranslator
from datetime import datetime
import urllib.parse
from json import dumps, loads
from shutil import copy2

In [5]:
os.chdir('/content/drive/MyDrive/research-similarity/Scraping')

In [6]:
def translate(text):
    return GoogleTranslator(source='vi', target='en').translate(text=text).replace(',', '').replace('\n', '')

def create_directories(base_url, categories, label='images'):
    # create the following dir struct; outputs > base website > categories
    base_dir = os.path.join('output', label, urlparse(base_url).netloc)
    if not os.path.exists(base_dir):
        os.makedirs(base_dir)

    for category, _ in categories:
        category_dir = os.path.join(base_dir, category)
        if not os.path.exists(category_dir):
            os.makedirs(category_dir)

    return base_dir

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
url = 'https://vnexpress.net/kinh-doanh/chung-khoan'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'lxml')

In [7]:
def get_articles_links(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml')
    main_content = soup.find('div', id='automation_TV0')
    articles_links = []

    for article in main_content.find_all('article'):
        a_tag = article.find('a')
        if a_tag:
            link = a_tag['href']
            if link not in articles_links:
                articles_links.append(link)

    return articles_links

def article_scrapper(url):
    articles_links = get_articles_links(url)
    data = []

    def helper_scrapper(url):
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'lxml')
        try:
            headline = translate(soup.find('h1').text)
            datetime_str = translate(soup.find('span', class_='date').text)[:-8]
            time = datetime.strptime(datetime_str, "%A %B %d %Y %H:%M")
            article_content = soup.find('article', class_='fck_detail')

            images = []

            for img in article_content.find_all('img'):
                img_alt = translate(img.get('alt', ''))  # Safely get the 'alt' attribute
                img_src = img.get('src', '')  # Safely get the 'src' attribute

                if (img_alt, img_src) not in images:
                    images.append((img_alt, img_src))

            return headline, time, images
        except:
            return None

    for url in articles_links:
        result = helper_scrapper(url)
        if result is not None and len(result[2]):
            data.append(result)

    return data

In [ ]:
get_articles_links(url='https://vnexpress.net/the-thao/tennis')

['https://vnexpress.net/alcaraz-thua-tran-ra-quan-laver-cup-4795367.html',
 'https://vnexpress.net/federer-du-doan-djokovic-doat-australia-mo-rong-4794853.html',
 'https://vnexpress.net/sinner-khong-duoc-chon-du-laver-cup-4794652.html',
 'https://vnexpress.net/khi-djokovic-giu-the-dien-cho-doi-thu-4793275.html',
 'https://vnexpress.net/alcaraz-bi-hoai-nghi-ve-dang-cap-so-voi-big-3-4793552.html',
 'https://vnexpress.net/djokovic-tap-trung-cho-serbia-o-cuoi-su-nghiep-4793247.html',
 'https://vnexpress.net/alcaraz-giup-tay-ban-nha-vao-tu-ket-davis-cup-4792982.html',
 'https://vnexpress.net/djokovic-thang-nadal-federer-nam-2011-la-buoc-ngoat-su-nghiep-4792724.html',
 'https://vnexpress.net/nadal-rut-khoi-laver-cup-4792465.html',
 'https://vnexpress.net/chu-nadal-vi-federer-nhu-messi-4792322.html',
 'https://vnexpress.net/alcaraz-giup-tay-ban-nha-thang-hai-tran-davis-cup-4791993.html',
 'https://vnexpress.net/sinner-duoc-ky-vong-se-khien-djokovic-giai-nghe-4791895.html',
 'https://vnexpress

In [ ]:
url = 'https://vnexpress.net/khi-djokovic-giu-the-dien-cho-doi-thu-4793275.html'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'lxml')

In [ ]:
soup.find('article', class_='fck_detail').find_all('img')[0].get('src')

'https://iv1cdn.vnecdn.net/thethao/images/web/2024/09/16/khi-phach-nghia-hiep-cua-djokovic-1726453657.jpg?w=0&h=0&q=100&dpr=1&fit=crop&s=2ppL9W7U8NrGuW4cWVt54g'

In [8]:
def get_latest_articles(data, n=10):
    seen_headlines = set()
    unique_data = []

    for record in data:
        headline = record[0]
        if headline not in seen_headlines:
            seen_headlines.add(headline)
            unique_data.append(record)

    return sorted(unique_data, key=lambda x: x[1], reverse=True)[:n]

def download_image(img_url, save_dir, img_name):
    try:
        if not img_url.startswith('data:'):
            response = requests.get(img_url)
            img_data = response.content
            img = Image.open(BytesIO(img_data))
            width, height = img.size

            # Only save images larger than 100x100 pixels
            if width >= 100 and height >= 100:
                with open(os.path.join(save_dir, img_name), 'wb') as img_file:
                    img_file.write(img_data)
    except:
        pass

def download_images(category_url, save_dir, data):

    with open(os.path.join(save_dir, 'labels.csv'), 'w') as f:
        f.write('image number,alt,article_heading\n')

    records = []

    # parallising the downloads to make it faster
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = []
        headlines = []
        for x, tuple in enumerate(data):
            headline, _, images_list = tuple
            for i, img in enumerate(images_list):
                alt_txt, img_url = img
                if img_url and not img_url.startswith('data:'):
                    img_url = urljoin(category_url, img_url)
                    combined_str = f"{alt_txt}{headline}".encode()
                    img_name = f'image_{x+1}_{i+1}.jpg'
                    records.append(f'{img_name},{alt_txt.replace(",", "")},{headline.replace(",", "")}\n')
                    futures.append(executor.submit(download_image, img_url, save_dir, img_name))

        with open(os.path.join(save_dir, 'labels.csv'), 'a') as f:
            f.writelines(records)

        for future in as_completed(futures):
            future.result()


In [10]:
categories = [
    ("Tennis", "https://vnexpress.net/the-thao/tennis"),
    ("Football", "https://vnexpress.net/bong-da"),
    ("Celebrity", "https://vnexpress.net/giai-tri/gioi-sao"),
    ("Movie", "https://vnexpress.net/giai-tri/phim"),
    ("Music", "https://vnexpress.net/giai-tri/nhac"),
    ("Fashion", "https://vnexpress.net/giai-tri/thoi-trang"),
    ("Beauty", "https://vnexpress.net/giai-tri/lam-dep"),
    ("Fine Arts", "https://vnexpress.net/giai-tri/san-khau-my-thuat"),
    ("Law", "https://vnexpress.net/phap-luat/ho-so-pha-an"),
    ("Health", "https://vnexpress.net/suc-khoe/tin-tuc"),
    ("Destination", "https://vnexpress.net/du-lich/diem-den"),
    ("Cuisine", "https://vnexpress.net/du-lich/am-thuc"),
    ("Footprint", "https://vnexpress.net/du-lich/dau-chan"),
    ("Technology", "https://vnexpress.net/so-hoa/cong-nghe"),
    ("Product", "https://vnexpress.net/so-hoa/san-pham"),
    ("Car", "https://vnexpress.net/oto-xe-may/thi-truong"),
    ("Life", "https://vnexpress.net/doi-song/to-am"),
    ("Military", "https://vnexpress.net/the-gioi/quan-su"),
    ("Politics", "https://vnexpress.net/thoi-su/chinh-tri"),
    ("People", "https://vnexpress.net/thoi-su/dan-sinh"),
    ("Business", "https://vnexpress.net/kinh-doanh/quoc-te"),
    ("Stock", "https://vnexpress.net/kinh-doanh/chung-khoan")
]

base_url = 'https://vnexpress.net/'

base_dir = create_directories(base_url, categories)

In [11]:
for category, category_url in tqdm(categories, desc='Downloading images for every category'):
    category_dir = os.path.join(base_dir, category)
    try:
        data = article_scrapper(category_url)
        download_images(category_url, category_dir, get_latest_articles(data))
    except:
        print(category)
        continue

In [15]:
def create_file(base_dir):
    for category in os.listdir(base_dir):
        try:
            df = pd.read_csv(f'{base_dir}{category}/labels.csv')
            n, _ = df.shape

            pairs = []

            for i in range(n):
                for j in range(i + 1, n):
                    article_1 = int(re.search(r'\d+(?=_|$)',list(df.iloc[i])[0]).group())
                    article_2 = int(re.search(r'\d+(?=_|$)',list(df.iloc[j])[0]).group())
                    if article_1 != article_2:
                        pairs.append((list(df.iloc[i]), list(df.iloc[j])))

            with open(f'{base_dir.split("/")[2]}_pairs_{category}.csv', 'w') as f:
                for i, pair in enumerate(pairs):
                    p1, p2 = pair
                    img1, alt1, headline1 = p1
                    img2, alt2, headline2 = p2
                    f.write(f'{i+1},{headline1},{headline2}\n')
                    f.write(f',https://raw.githubusercontent.com/ayainfida/news-scrapper/main/output/images/vnexpress.net/{urllib.parse.quote(category)}/{img1},https://raw.githubusercontent.com/ayainfida/news-scrapper/main/output/images/vnexpress.net/{urllib.parse.quote(category)}/{img2}\n')
                    f.write(f',{alt1},{alt2}\n')

        except:
            continue

In [16]:
create_file('output/images/vnexpress.net/')

In [17]:
num_pairs = []

for file in os.listdir():
    if file.startswith('vnexpress.net_pairs'):
        try:
            df = pd.read_csv(file)
            num_pairs.append((int(list(df.iloc[-3])[0]), file))
        except:
            continue

In [18]:
sorted_array = sorted(num_pairs, key=lambda x: x[0])
sorted_array

[(45, 'vnexpress.net_pairs_Tennis.csv'),
 (45, 'vnexpress.net_pairs_Stock.csv'),
 (63, 'vnexpress.net_pairs_Politics.csv'),
 (74, 'vnexpress.net_pairs_Business.csv'),
 (85, 'vnexpress.net_pairs_Health.csv'),
 (97, 'vnexpress.net_pairs_Technology.csv'),
 (127, 'vnexpress.net_pairs_Football.csv'),
 (142, 'vnexpress.net_pairs_Car.csv'),
 (195, 'vnexpress.net_pairs_Life.csv'),
 (212, 'vnexpress.net_pairs_Military.csv'),
 (215, 'vnexpress.net_pairs_Law.csv'),
 (236, 'vnexpress.net_pairs_Footprint.csv'),
 (298, 'vnexpress.net_pairs_Product.csv'),
 (324, 'vnexpress.net_pairs_Fine Arts.csv'),
 (342, 'vnexpress.net_pairs_Celebrity.csv'),
 (431, 'vnexpress.net_pairs_Fashion.csv'),
 (451, 'vnexpress.net_pairs_Music.csv'),
 (458, 'vnexpress.net_pairs_People.csv'),
 (459, 'vnexpress.net_pairs_Destination.csv'),
 (481, 'vnexpress.net_pairs_Movie.csv'),
 (515, 'vnexpress.net_pairs_Beauty.csv'),
 (574, 'vnexpress.net_pairs_Cuisine.csv')]